In [ ]:
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

**importing datasets**

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths to your dataset
train_dir = '/content/drive/MyDrive/train'
validation_dir = '/content/drive/MyDrive/validation'

In [ ]:
# Image data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

Found 1585 images belonging to 3 classes.
Found 395 images belonging to 3 classes.


**Model Building**

In [ ]:
# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # You can also monitor 'val_accuracy'
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # Print messages when stopping
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model with early stopping
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,
    callbacks=[early_stopping]  # Include the early stopping callback
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 474s 10s/step - accuracy: 0.5526 - loss: 1.6655 - val_accuracy: 0.6901 - val_loss: 0.7082
Epoch 2/50
 1/49 ━━━━━━━━━━━━━━━━━━━━ 1:30 2s/step - accuracy: 0.6250 - loss: 0.7602

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 152ms/step - accuracy: 0.6250 - loss: 0.7602 - val_accuracy: 0.7266 - val_loss: 0.6931
Epoch 3/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.6598 - loss: 0.7612 - val_accuracy: 0.7318 - val_loss: 0.7230
Epoch 4/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 188ms/step - accuracy: 0.6562 - loss: 0.7263 - val_accuracy: 0.7240 - val_loss: 0.7189
Epoch 5/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.6872 - loss: 0.6919 - val_accuracy: 0.7344 - val_loss: 0.6335
Epoch 6/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 12s 195ms/step - accuracy: 0.6250 - loss: 0.9097 - val_accuracy: 0.7318 - val_loss: 0.6352
Epoch 7/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.6574 - loss: 0.7076 - val_accuracy: 0.7344 - val_loss: 0.6735
Epoch 8/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 16s 300ms/step - accuracy: 0.6250 - loss: 0.9185 - val_accuracy: 0.7344 - val_loss: 0.6608
Epoch 9/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.7001 - loss: 0.6941 - val_accuracy: 0.7344 - val_l

**Model evaluation**

In [ ]:
# Save the model
model.save('my_model.h5')  # Save model in HDF5 format

In [ ]:
# Get the absolute path of the model file
model_path = os.path.abspath('my_model.h5')

# Display the path
print(f"The model is saved at: {model_path}")

The model is saved at: /content/my_model.h5


In [ ]:
from google.colab import files

# Download the model file
files.download('/content/my_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>